#### A Quick Demo for the Mixture of Joint Distributions (MJD) Algorithm with a One-Hidden-Layer Neual Network

In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchmin import minimize

In [2]:
# define the one-hidden-layer neural network model
class NeuralNet(nn.Module):
    def __init__(self, input_size=1000, hidden_size=100, output_size=10):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(in_features=input_size, out_features=hidden_size, bias=True)
        self.fc2 = nn.Linear(in_features=hidden_size, out_features=output_size, bias=True)
    def forward(self, X):
        FX = F.relu(self.fc1(X)) # hidden layer activation features
        prob = F.softmax(self.fc2(FX), dim=1) # probability output
        return FX, prob

In [3]:
# wrap the Mixture of Joint Distributions (MJD) algorithm as a class, following the sklearn style
class MJD:
    """
    In the training procedure, the total batch size per iteration is: batch_size * num_class * num_domain
    For instance, if there are 5 domains with 68 classes in each domain, 
    then batch_size=4 means drawing 4 samples from every class in every domain,
    resulting in 4*68*5 samples in the total batch size.
    """
    def __init__(self, input_size=1024, hidden_size=512, output_size=68, seed=1000, device=torch.device('cpu'),
                 epoch_pretrain=200, epoch=200, lamda=1, epsilon=1e-3, batch_size=4, lr=1e-3, log=False):
        args_values = locals()
        args_values.pop("self")
        for arg,value in args_values.items():
            setattr(self, arg, value)
            
    def fit(self, X_list, y_list, Xt, yt):
        class_labels = np.unique(y_list[0])
        n, c = len(X_list), len(class_labels) # number of source domains, number of classes
        # generate random target labels
        yt_rand = np.random.choice(a=class_labels, size=len(Xt), replace=True, p=1.0 * np.ones(c) / c) 
        X_list.append(Xt), y_list.append(yt_rand)
        
        # define the neural network instance and the optimizer
        torch.manual_seed(self.seed)
        net = NeuralNet(input_size=self.input_size, hidden_size=self.hidden_size, output_size=self.output_size).to(self.device)
        optimizer = optim.SGD(params=net.parameters(), lr=self.lr, momentum=0.9)
        
        #=====pretrain the network to estimate the pseudo target labels=======
        print('Pretraining...')
        for epoch in range(self.epoch_pretrain):
            dataset_loaders, l = [], 0
            for X, y in zip(X_list, y_list):
                for i, counts in zip(*np.unique(y, return_counts=True)):
                    dataset = np.hstack((X[y==i], y[y==i][:,None], l * np.ones((counts,1))))
                    dataset_loaders.append(torch.utils.data.DataLoader(dataset=torch.tensor(dataset),
                                                                       batch_size=self.batch_size, shuffle=True, drop_last=False))
                l = l + 1 # source domain labels {0, ..., n-1}, target domain label n
            
            log_loss, m_log_loss = 0.0, 0.0
            for batches in zip(*dataset_loaders):
                Xyl = torch.cat(batches, dim=0)
                X, y, l = Xyl[:,:-2].to(self.device,torch.float32), Xyl[:,-2].to(self.device,torch.int64), Xyl[:,-1].to(self.device,torch.int64)

                FX, prob = net(X)
                negative_log = 0.0
                # weights for the source domains are identical in the pretraining procedure
                for i in range(n):
                    negative_log += -1.0 / n * torch.mean(torch.sum(torch.log(prob[l==i]) * F.one_hot(y[l==i], c), dim=1))
                loss = negative_log
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                log_loss += negative_log.item() * len(X[l!=n])
                m_log_loss += len(X[l!=n])

            with torch.no_grad():
                Xt, yt = torch.as_tensor(Xt, dtype=torch.float32, device=self.device), torch.as_tensor(yt, dtype=torch.int64, device=self.device)    
                yt_hat = torch.argmax(net(Xt)[1],dim=1)
                correct = torch.sum((yt_hat == yt)).item()
                m_test = len(yt)
            
            if True == self.log:
                print('epoch ',epoch, ', log loss ',  "{:.5f}".format(log_loss / m_log_loss), 
                      ', test acc. ', "{:.5f}".format((correct / m_test) * 100)) 
        #========================================================
    
        #=====train the MSDA network====================================
        print('Training...')
        for epoch in range(self.epoch):
            # update the pseudo target labels every epoch
            y_list.pop()
            y_list.append(yt_hat.cpu().numpy())
            dataset_loaders, l = [], 0
            for X, y in zip(X_list, y_list):
                for i, counts in zip(*np.unique(y, return_counts=True)):
                    dataset = np.hstack((X[y==i], y[y==i][:,None], l * np.ones((counts,1))))
                    dataset_loaders.append(torch.utils.data.DataLoader(dataset=torch.tensor(dataset),
                                                                       batch_size=self.batch_size, shuffle=True, drop_last=False))
                l = l + 1 # source domain labels {0, ..., n-1}, target domain label n
            
            log_loss, m_log_loss = 0.0, 0.0
            # n + 1 batches of identical size are drawn from the n + 1 source and target datasets
            # each batch contains the same number of samples from each class
            for batches in zip(*dataset_loaders):
                Xyl = torch.cat(batches, dim=0)
                X, y, l = Xyl[:,:-2].to(self.device, torch.float32), Xyl[:,-2].to(self.device, torch.int64), Xyl[:,-1].to(self.device, torch.int64)
                
                # compute the Gaussian kernel width
                pairwise_dist = torch.cdist(X, X, p=2)**2 
                sigma = torch.median(pairwise_dist[pairwise_dist!=0]) 
                
                # compute the product kernel matrix
                FX, prob = net(X)
                FX_norm = torch.sum(FX ** 2, axis = -1)
                K = torch.exp(-(FX_norm[:,None] + FX_norm[None,:] - 2 * torch.matmul(FX, FX.t())) / sigma) # feature kernel matrix     
                Deltay = torch.as_tensor(y[:,None]==y, dtype=torch.float64, device=FX.device) # label kernel matrix  
                P = torch.as_tensor(K * Deltay, dtype=torch.double) # product kernel matrix
                
                H = 1.0 / len(P[l==n]) * torch.matmul((P[l==n]).t(), P[l==n])
                invM = torch.inverse(H + self.epsilon * torch.eye(len(P), device=FX.device))
                
                # optimize the mixing weights
                def ObjAlpha(alpha):
                    alpha = torch.softmax(alpha, dim=0)
                    negative_log, b = 0.0, 0.0
                    for i in range(n):
                        negative_log += -alpha[i] * torch.mean(torch.sum(torch.log(prob[l==i]) * F.one_hot(y[l==i], c), dim=1))       
                        b += alpha[i] * torch.mean(P[l==i], axis=0)
                    theta = torch.matmul(invM, b)
                    # the estimated Pearson chi2 divergence as a loss of alpha
                    chi2 = 2 * torch.matmul(b, theta) - torch.matmul(theta, torch.matmul(H, theta)) - 1.0
                    return negative_log + self.lamda * chi2
                alpha0 = 1.0 * torch.ones(n, device=self.device) / n 
                result = minimize(ObjAlpha, alpha0, method='l-bfgs', max_iter=50)
                alpha = torch.softmax(result.x, dim=0)
                
                negative_log = 0.0
                for i in range(n):
                    negative_log += -alpha[i] * torch.mean(torch.sum(torch.log(prob[l==i]) * F.one_hot(y[l==i], c), dim=1))
                b = 0.0
                for i in range(n):     
                    b += alpha[i] * torch.mean(P[l==i], axis=0)
                theta = torch.matmul(invM, b)
                # the estimated Pearson chi2 divergence as a loss of the feature extractor
                chi2 = 2 * torch.matmul(b, theta) - torch.matmul(theta, torch.matmul(H, theta)) - 1.0
                loss = negative_log + self.lamda * chi2
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                log_loss += negative_log.item() * len(X[l!=n])
                m_log_loss += len(X[l!=n])

            with torch.no_grad():
                Xt, yt = torch.as_tensor(Xt, dtype=torch.float32, device=self.device), torch.as_tensor(yt, dtype=torch.int64, device=self.device)    
                yt_hat = torch.argmax(net(Xt)[1],dim=1)
                correct = torch.sum((yt_hat == yt)).item()
                m_test = len(yt)
                
            if True == self.log:
                print('epoch ',epoch, ', log loss ',  "{:.5f}".format(log_loss / m_log_loss), 
                      ', chi2 loss ', "{:.5f}".format(chi2.item()), 
                      ', total loss ', "{:.5f}".format(log_loss / m_log_loss + self.lamda * chi2.item()),
                      ', test acc. ', "{:.5f}".format((correct / m_test) * 100)) 
            #========================================================     
        self.net = net # save the network

    def score(self, Xt, yt):
        with torch.no_grad():
            Xt, yt = torch.as_tensor(Xt, dtype=torch.float32,device=self.device), torch.as_tensor(yt, dtype=torch.int64,device=self.device)    
            pred = torch.argmax(self.net(Xt)[1],dim=1)
            correct = torch.sum((pred == yt)).item()
            m_test = len(yt)
        return (correct / m_test) * 100

In [4]:
import numpy as np
import pandas as pd
import scipy.io as sio
import numpy.linalg as la
from sklearn.preprocessing import scale,LabelEncoder

In [5]:
def readData(tg, domains):
    data = sio.loadmat('PIE/' + tg + '.mat')
    Xt, yt = data['fea'].astype(np.float64), data['gnd'].ravel()
    yt = LabelEncoder().fit(yt).transform(yt).astype(np.float64)
    Xt = scale(Xt / Xt.sum(axis=1,keepdims=True))
    
    Xs_list, ys_list = [], []
    for sc in domains:
        if sc != tg:
            data = sio.loadmat('PIE/' + sc + '.mat')
            Xs, ys = data['fea'].astype(np.float64), data['gnd'].ravel()
            ys = LabelEncoder().fit(ys).transform(ys).astype(np.float64)
            Xs = scale(Xs / Xs.sum(axis=1,keepdims=True))
            Xs_list.append(Xs), ys_list.append(ys)        
    
    return Xs_list, ys_list, Xt, yt

domains = ['C05', 'C07', 'C09', 'C27', 'C29']

In [6]:
DEVICE = torch.device('cpu') # 'cuda:0'
Xs_list, ys_list, Xt, yt = readData('C05', domains)
instance = MJD(input_size=1024, hidden_size=512, output_size=68, seed=0, device=DEVICE,
                         epoch_pretrain=30, epoch=50, lamda=10, epsilon=1e-3, batch_size=3, lr=1e-2, log=True)
instance.fit(Xs_list, ys_list, Xt, yt)
instance.score(Xt, yt)

Pretraining...
epoch  0 , log loss  4.22435 , test acc.  3.21128
epoch  1 , log loss  4.10045 , test acc.  9.39376
epoch  2 , log loss  3.92304 , test acc.  16.53661
epoch  3 , log loss  3.72881 , test acc.  22.38896
epoch  4 , log loss  3.52655 , test acc.  26.53061
epoch  5 , log loss  3.31548 , test acc.  31.66267
epoch  6 , log loss  3.09318 , test acc.  37.30492
epoch  7 , log loss  2.87622 , test acc.  42.10684
epoch  8 , log loss  2.65817 , test acc.  47.98920
epoch  9 , log loss  2.43428 , test acc.  52.40096
epoch  10 , log loss  2.22172 , test acc.  56.48259
epoch  11 , log loss  2.00899 , test acc.  59.84394
epoch  12 , log loss  1.80526 , test acc.  63.02521
epoch  13 , log loss  1.63748 , test acc.  64.97599
epoch  14 , log loss  1.46537 , test acc.  66.62665
epoch  15 , log loss  1.32250 , test acc.  67.25690
epoch  16 , log loss  1.19497 , test acc.  68.27731
epoch  17 , log loss  1.06854 , test acc.  69.11765
epoch  18 , log loss  0.96872 , test acc.  69.77791
epoch  19

94.38775510204081